# serverless endpoint

https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html

In [1]:
# !pip3 install sagemaker
#!pip3 install --upgrade botocore

In [2]:
import json

from abcli import fullname
from abcli import string

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{fullname()}, built on {string.pretty_date()}")

abcli-7.2554.1-unknown, built on 09 December 2023, 17:10:08


---

## creating the model

In [3]:
model_name = "model-2023-12-03-11-24-39-75649"
model_url = f"s3://kamangir/bolt/{model_name}.tar.gz"

In [4]:
import boto3
import sagemaker
region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)

Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
# Role to give SageMaker permission to access AWS services.
# https://github.com/aws-solutions-library-samples/guidance-for-training-an-aws-deepracer-model-using-amazon-sagemaker/issues/47#issuecomment-607238430
try:
    sagemaker_role = sagemaker.get_execution_role()
except ValueError:
    logger.info("sagemaker_role: defaulting to local.")
    iam = boto3.client('iam')
    sagemaker_role = iam.get_role(RoleName='Sagemaker-Access')['Role']['Arn']

sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name shamim to get Role path.
sagemaker_role: defaulting to local.


In [6]:
# Get container image (prebuilt example)
from sagemaker import image_uris
container = image_uris.retrieve("xgboost", region, "0.90-1")

Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.


In [7]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/list_models.html
response = client.list_models(
    NameContains=model_name
)

model_found = bool(response["Models"])
if model_found:
    logger.info("model already exists, will delete first.")

model already exists, will delete first.


In [8]:
if model_found:
    # delete the model
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/delete_model.html#
    response = client.delete_model(
        ModelName=model_name
    )
    logger.info(json.dumps(response,indent=4))

{
    "ResponseMetadata": {
        "RequestId": "22795b45-f7da-4f13-84fb-b44480589305",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "22795b45-f7da-4f13-84fb-b44480589305",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "0",
            "date": "Sun, 10 Dec 2023 01:10:10 GMT"
        },
        "RetryAttempts": 0
    }
}


In [9]:
# Create model
response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    Containers = [{
        "Image": container,
        "Mode": "SingleModel",
        "ModelDataUrl": model_url,
    }]
)

logger.info(json.dumps(response,indent=4))

{
    "ModelArn": "arn:aws:sagemaker:ca-central-1:120429650996:model/model-2023-12-03-11-24-39-75649",
    "ResponseMetadata": {
        "RequestId": "30dafcb4-7816-4edb-b6fa-ca7b4939ebc4",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "30dafcb4-7816-4edb-b6fa-ca7b4939ebc4",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "96",
            "date": "Sun, 10 Dec 2023 01:10:11 GMT"
        },
        "RetryAttempts": 0
    }
}


In [10]:
response = client.list_models(NameContains=model_name)
if response["Models"]:
    logger.info(response["Models"])
else:
    logger.error("model was not created.")
    assert False

[{'ModelName': 'model-2023-12-03-11-24-39-75649', 'ModelArn': 'arn:aws:sagemaker:ca-central-1:120429650996:model/model-2023-12-03-11-24-39-75649', 'CreationTime': datetime.datetime(2023, 12, 9, 17, 10, 12, 255000, tzinfo=tzlocal())}]


## Creating the endpoint configuration

In [11]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/list_endpoint_configs.html
response = client.list_endpoint_configs(NameContains=model_name)

endpoint_config_found = bool(response["EndpointConfigs"])
if endpoint_config_found:
    logger.info("endpoint config already exists, will delete first.")

endpoint config already exists, will delete first.


In [12]:
if endpoint_config_found:
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/delete_endpoint_config.html
    response = client.delete_endpoint_config(EndpointConfigName=model_name)
    logger.info(json.dumps(response,indent=4))    

{
    "ResponseMetadata": {
        "RequestId": "9a1cf6bb-3ff9-413b-b9b1-23e2ab74dc3c",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "9a1cf6bb-3ff9-413b-b9b1-23e2ab74dc3c",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "0",
            "date": "Sun, 10 Dec 2023 01:10:11 GMT"
        },
        "RetryAttempts": 0
    }
}


In [13]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html#serverless-endpoints-create-config

response = client.create_endpoint_config(
   EndpointConfigName=model_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 20,
                #"ProvisionedConcurrency": 10,
            }
        } 
    ]
)

logger.info(json.dumps(response,indent=4))

{
    "EndpointConfigArn": "arn:aws:sagemaker:ca-central-1:120429650996:endpoint-config/model-2023-12-03-11-24-39-75649",
    "ResponseMetadata": {
        "RequestId": "39f10528-8a1d-48a8-9610-3540a8b39ddd",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "39f10528-8a1d-48a8-9610-3540a8b39ddd",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "115",
            "date": "Sun, 10 Dec 2023 01:10:12 GMT"
        },
        "RetryAttempts": 0
    }
}


In [14]:
response = client.list_endpoint_configs(NameContains=model_name)

if response["EndpointConfigs"]:
    logger.info(response["EndpointConfigs"])
else:
    logger.error("endpoint config was not created.")
    assert False

[{'EndpointConfigName': 'model-2023-12-03-11-24-39-75649', 'EndpointConfigArn': 'arn:aws:sagemaker:ca-central-1:120429650996:endpoint-config/model-2023-12-03-11-24-39-75649', 'CreationTime': datetime.datetime(2023, 12, 9, 17, 10, 13, 110000, tzinfo=tzlocal())}]


---

In [15]:
# END